In [2]:
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob
   
def find_four_points(img,ratio,show):
    img= cv2.medianBlur(img,5)
    img = cv2.GaussianBlur(img, (3,3), 5)
    detect_area_size = 340 / ratio
    detect_area_size = int(detect_area_size)
    detect_area = [img[:detect_area_size,:detect_area_size],#左上
                   img[:detect_area_size,-detect_area_size:],#右上
                   img[-detect_area_size:,:detect_area_size],#左下
                   img[-detect_area_size:,-detect_area_size:]]#右下

    detect_start = [[0,0],
                   [new_width-detect_area_size,0],
                   [0,new_height-detect_area_size],
                   [new_width-detect_area_size,new_height-detect_area_size]]
    
    circles = []
    for area in detect_area:
        circle = cv2.HoughCircles(area,cv2.HOUGH_GRADIENT,1,50,
                                param1=20,param2=4,minRadius=5,maxRadius=30)
        assert circle.shape[0] == 1
        circle = np.uint16(np.around(circle))
        circles.append(circle[0,0,:2])

    for i in range(4):
        assert circles[i][0] != 0 and circles[i][1] != 0
        circles[i][0] += detect_start[i][0]
        circles[i][1] += detect_start[i][1]
    circles = np.asarray(circles,dtype=np.uint32)

    if show:
        for i in circles:
            cv2.circle(img,(i[0],i[1]),30,(0,0,255),30)
        cv2.imshow('detected circles',img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    return circles   
            
    
    
def min_rectangle(circles):
    rect_0 = [min(circles[:,0]),min(circles[:,1])]
    rect_1 = [max(circles[:,0]),min(circles[:,1])]
    rect_2 = [min(circles[:,0]),max(circles[:,1])]
    rect_3 = [max(circles[:,0]),max(circles[:,1])]
    return np.array([rect_0,rect_1,rect_2,rect_3],dtype=np.int32)


i = 0
for name in glob.glob('origine_images\*'):
    img = cv2.imread(name,1)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    height = img.shape[0]
    width = img.shape[1]
    ratio = 1
    new_height = int(height/ratio)
    new_width = int(width/ratio)
    img = cv2.resize(img,(new_width,new_height))
#     cv2.imshow('img',img)
#     cv2.waitKey(0)

    circles = find_four_points(img,ratio,0)
    min_rect = min_rectangle(circles)

    #PerspectiveTransform仿射变幻
    M = cv2.getPerspectiveTransform(np.float32(circles), np.float32(min_rect))
    img = cv2.warpPerspective(img[min_rect[0,1]:min_rect[2,1],min_rect[0,0]:min_rect[1,0]], M, 
                                          (min_rect[0,1]-min_rect[2,1],min_rect[0,0]-min_rect[1,0]), cv2.INTER_LINEAR)
    img = cv2.resize(img,(int(2090/ratio),int(2990/ratio)))
    
    #ensure the order of the papers 
    if np.mean(img[int(height*0.6):int(height*0.9),int(width*0.6):int(width*0.9)]) < 254 : 
        write_name = 'images/' + str(int(i/2)) + '-' + '0.jpg'
    else:
        write_name = 'images/' + str(int(i/2)) + '-' + '1.jpg'
    #cv2.imwrite(write_name,img)
    i += 1
#    print(write_name)
#     cv2.imshow('img',img)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

In [9]:
img1 = cv2.imread('images/0-0.jpg',1)
img2 = cv2.imread('images/0-1.jpg',1)

height = img1.shape[0]
width = img1.shape[1]

img1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
img2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

img1_head = img[:int(height*0.13),:]
img1_body_left = img1[int(height*0.13):,:int(width*0.5)]
img1_body_right = img1[int(height*0.13):,int(width*0.5):]

img2_left = img2[:,:int(width*0.5)]
img2_right = img2[:,int(width*0.5):]


img_concat = np.concatenate((img1_body_left,img1_body_right,img2_left,img2_right),axis=0)
print(img_concat.shape)
i = 44
#cv2.imshow('img',img_concat[1014+int(36*0.75*i):1068+int(36*0.75*i),:])#选项卡边长36像素
ret, thresh = cv2.threshold(img, 127, 255, 0)
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
print(contours)
cv2.drawContours(img, contours, -1, (0,255,0), 30)
#ratio = 3
#new_height = int(height/ratio)
#new_width = int(width/ratio)
#img = cv2.resize(img,(new_width,new_height))
#img= cv2.medianBlur(img,5)
#cv2.imshow('img',img)
cv2.waitKey(0)

(11184, 1045)
[array([[[ 280, 2684]],

       [[ 282, 2684]]], dtype=int32), array([[[ 520, 2683]],

       [[ 519, 2684]],

       [[ 518, 2684]],

       [[ 522, 2684]],

       [[ 523, 2683]],

       [[ 525, 2683]]], dtype=int32), array([[[ 289, 2683]],

       [[ 288, 2684]],

       [[ 284, 2684]],

       [[ 285, 2685]],

       [[ 286, 2685]],

       [[ 287, 2686]],

       [[ 288, 2686]],

       [[ 289, 2687]],

       [[ 291, 2687]],

       [[ 292, 2688]],

       [[ 294, 2688]],

       [[ 295, 2687]],

       [[ 296, 2687]],

       [[ 297, 2686]],

       [[ 298, 2686]],

       [[ 298, 2685]],

       [[ 297, 2685]],

       [[ 296, 2684]],

       [[ 295, 2684]],

       [[ 294, 2683]]], dtype=int32), array([[[ 178, 2681]],

       [[ 177, 2682]],

       [[ 176, 2682]],

       [[ 176, 2683]],

       [[ 179, 2683]],

       [[ 180, 2682]],

       [[ 179, 2681]]], dtype=int32), array([[[1003, 2507]],

       [[ 997, 2513]],

       [[ 996, 2513]],

       [[ 995, 25

-1